In [20]:
import ccxt
import pandas as pd

# Initialize the exchanges
binance = ccxt.binance({'enableRateLimit': True})
kraken = ccxt.kraken({'enableRateLimit': True})

# Define the less mainstream cryptocurrency pair
symbol = 'LTC/ETH'  # Replace with the actual symbol, e.g., 'LTC/ETH'

# Function to fetch 5-min OHLCV data
def fetch_ohlcv(exchange, symbol):
    # Get 5min ticker information for the last 24 hours
    ohlcv = exchange.fetch_ohlcv(symbol, '5m', since=exchange.milliseconds() - (24 * 60 * 60 * 1000))
    df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)
    return df

# Fetch data from both exchanges
df_binance = fetch_ohlcv(binance, symbol)
df_kraken = fetch_ohlcv(kraken, symbol)
print(df_binance)
print(df_kraken)

                        open     high      low    close   volume
timestamp                                                       
2023-11-10 18:40:00  0.03461  0.03467  0.03461  0.03467    5.434
2023-11-10 18:45:00  0.03468  0.03475  0.03467  0.03473  353.406
2023-11-10 18:50:00  0.03475  0.03478  0.03475  0.03478    5.118
2023-11-10 18:55:00  0.03479  0.03486  0.03479  0.03485   33.448
2023-11-10 19:00:00  0.03486  0.03486  0.03481  0.03481   16.726
...                      ...      ...      ...      ...      ...
2023-11-11 18:15:00  0.03610  0.03610  0.03610  0.03610    0.046
2023-11-11 18:20:00  0.03609  0.03609  0.03603  0.03603   21.225
2023-11-11 18:25:00  0.03601  0.03602  0.03599  0.03599   49.641
2023-11-11 18:30:00  0.03598  0.03598  0.03591  0.03591   15.610
2023-11-11 18:35:00  0.03591  0.03591  0.03591  0.03591    0.000

[288 rows x 5 columns]
                        open     high      low    close   volume
timestamp                                                       
2

In [21]:
# Paramètres de simulation
capital_alloc = 100  # Le capital alloué pour chaque opportunité d'arbitrage
transaction_fee_binance = 0.001  # Frais de transaction pour Binance
transaction_fee_kraken = 0.001   # Frais de transaction pour Kraken

# Fonction pour calculer le slippage basé sur la volatilité intrajournalière
def calculate_slippage(row):
    high_low_range_binance = row['high_binance'] - row['low_binance']
    high_low_range_kraken = row['high_kraken'] - row['low_kraken']
    
    # Estimation du slippage comme étant un pourcentage de la fourchette high-low
    slippage_binance = high_low_range_binance * 0.2  # Exemple avec 20%
    slippage_kraken = high_low_range_kraken * 0.2
    
    return slippage_binance, slippage_kraken

In [22]:
# Fonction pour calculer le gain potentiel d'une opportunité d'arbitrage
def calculate_arbitrage_profit(row):
    slippage_binance, slippage_kraken = calculate_slippage(row)

    # Calcul du volume que l'on peut acheter avec le capital alloué
    volume_binance = capital_alloc / (row['open_binance'] + slippage_binance)
    volume_kraken = capital_alloc / (row['open_kraken'] + slippage_kraken)

    # Calcul des revenus basés sur le prix de clôture moins le slippage estimé
    revenue_binance = (volume_binance * row['close_binance']) * (1 - transaction_fee_binance)
    revenue_kraken = (volume_kraken * row['close_kraken']) * (1 - transaction_fee_kraken)

    # Calcul du profit en supposant que l'on achète sur Kraken et vend sur Binance
    if row['open_binance'] > row['open_kraken']:
        profit = revenue_binance - (capital_alloc * (1 + transaction_fee_kraken))
    else:
        profit = revenue_kraken - (capital_alloc * (1 + transaction_fee_binance))

    return profit

In [23]:
# Appliquer la fonction de calcul à chaque ligne pour les opportunités d'arbitrage identifiées
df_arbitrage['profit'] = df_arbitrage.apply(calculate_arbitrage_profit, axis=1)

# Afficher les opportunités d'arbitrage avec le profit estimé
print(df_arbitrage[['open_binance', 'open_kraken', 'close_binance', 'close_kraken', 'profit']])

                     open_binance  open_kraken  close_binance  close_kraken  \
timestamp                                                                     
2023-11-10 17:55:00       0.03455      0.03457        0.03453       0.03454   
2023-11-10 18:00:00       0.03450      0.03454        0.03450       0.03454   
2023-11-10 18:05:00       0.03450      0.03454        0.03449       0.03454   
2023-11-10 18:10:00       0.03451      0.03454        0.03455       0.03454   
2023-11-10 18:15:00       0.03456      0.03459        0.03461       0.03464   
...                           ...          ...            ...           ...   
2023-11-11 17:30:00       0.03585      0.03581        0.03588       0.03581   
2023-11-11 17:35:00       0.03590      0.03581        0.03599       0.03581   
2023-11-11 17:40:00       0.03600      0.03601        0.03610       0.03601   
2023-11-11 17:45:00       0.03609      0.03601        0.03608       0.03601   
2023-11-11 17:50:00       0.03608      0.03601      

In [92]:
import ccxt
import pandas as pd
import random

# Get a list of all exchanges that support fetchOHLCV
exchanges_with_ohlcv = [exchange for exchange in ccxt.exchanges if getattr(ccxt, exchange)().has['fetchOHLCV']]

# Ensure we have at least two such exchanges
if len(exchanges_with_ohlcv) < 2:
    raise Exception("Not enough exchanges with fetchOHLCV support.")

# Select two random exchanges from the list of exchanges that support fetchOHLCV
selected_exchanges = random.sample(exchanges_with_ohlcv, 2)

# Initialize the selected exchanges with rate limit enabled
exchange1 = getattr(ccxt, selected_exchanges[0])({'enableRateLimit': True})
exchange2 = getattr(ccxt, selected_exchanges[1])({'enableRateLimit': True})

# exchange1 = ccxt.binance({'enableRateLimit': True})
# exchange2 = ccxt.kraken({'enableRateLimit': True})

print(f"Exchanges : {exchange1}/{exchange2}")
# Define the cryptocurrency pair
symbol = 'LTC/ETH'  # Replace with the actual symbol, e.g., 'LTC/ETH'

# Ensure both exchanges support the chosen symbol
if not exchange1.has['fetchOHLCV'] or not exchange2.has['fetchOHLCV']:
    raise Exception(f"One of the exchanges does not support OHLCV data fetching.")

def fetch_common_symbols(exchange1, exchange2):
    # Load markets for both exchanges
    exchange1.load_markets()
    exchange2.load_markets()

    # Find common symbols between both exchanges
    common_symbols = list(set(exchange1.symbols) & set(exchange2.symbols))

    # Raise an exception if there are no common symbols
    if not common_symbols:
        raise Exception(f"No common symbols found between {exchange1.id} and {exchange2.id}")

    # Select two random symbols from the list of common symbols
    selected_symbols = random.sample(common_symbols, 1)
    
    return selected_symbols

def fetch_ohlcv_for_common_symbols(exchange, symbols):
    dataframes = {}
    for symbol in symbols:
        # Get 5min ticker information for the last 24 hours for each symbol
        ohlcv = exchange.fetch_ohlcv(symbol, '5m', since=exchange.milliseconds() - (24 * 60 * 60 * 1000))
        df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        df.set_index('timestamp', inplace=True)
        dataframes[symbol] = df
    
    return dataframes

# Fetch OHLCV data for these symbols from both exchanges
common_symbols = fetch_common_symbols(exchange1, exchange2)[0]
df_exchange1 = fetch_ohlcv_for_common_symbols(exchange1, [common_symbols])[common_symbols]
df_exchange2 = fetch_ohlcv_for_common_symbols(exchange2, [common_symbols])[common_symbols]

df_arbitrage = pd.merge(df_exchange1, df_exchange2, on='timestamp', suffixes=('_exchange1', '_exchange2'))

# Add columns for high and low ranges for slippage calculation
df_arbitrage['high_exchange1'] = df_exchange1['high']
df_arbitrage['low_exchange1'] = df_exchange1['low']
df_arbitrage['high_exchange2'] = df_exchange2['high']
df_arbitrage['low_exchange2'] = df_exchange2['low']


# Paramètres de simulation
capital_alloc = 100  # Le capital alloué pour chaque opportunité d'arbitrage
transaction_fee_exchange1 = 0.001  # Frais de transaction pour exchange1
transaction_fee_exchange2 = 0.001   # Frais de transaction pour exchange2

# Fonction pour calculer le slippage basé sur la volatilité intrajournalière
def calculate_slippage(row):
    high_low_range_exchange1 = row['high_exchange1'] - row['low_exchange1']
    high_low_range_exchange2 = row['high_exchange2'] - row['low_exchange2']
    
    # Estimation du slippage comme étant un pourcentage de la fourchette high-low
    slippage_exchange1 = high_low_range_exchange1 * 0.2  # Exemple avec 20%
    slippage_exchange2 = high_low_range_exchange2 * 0.2
    
    return slippage_exchange1, slippage_exchange2

# Fonction pour calculer le gain potentiel d'une opportunité d'arbitrage
def calculate_arbitrage_profit(row):
    slippage_exchange1, slippage_exchange2 = calculate_slippage(row)

    # Calcul du volume que l'on peut acheter avec le capital alloué
    volume_exchange1 = capital_alloc / (row['open_exchange1'] + slippage_exchange1)
    volume_exchange2 = capital_alloc / (row['open_exchange2'] + slippage_exchange2)

    # Calcul des revenus basés sur le prix de clôture moins le slippage estimé
    revenue_exchange1 = (volume_exchange1 * row['close_exchange1']) * (1 - transaction_fee_exchange1)
    revenue_exchange2 = (volume_exchange2 * row['close_exchange2']) * (1 - transaction_fee_exchange2)

    # Calcul du profit en supposant que l'on achète sur exchange2 et vend sur exchange1
    if row['open_exchange1'] > row['open_exchange2']:
        profit = revenue_exchange1 - (capital_alloc * (1 + transaction_fee_exchange2))
    else:
        profit = revenue_exchange2 - (capital_alloc * (1 + transaction_fee_exchange1))

    return profit

# Calculate the profit for each row
df_arbitrage['profit'] = df_arbitrage.apply(calculate_arbitrage_profit, axis=1)
print(df_arbitrage[['open_exchange1', 'open_exchange2', 'close_exchange1', 'close_exchange2', 'profit']])

Exchanges : Mercado Bitcoin/Bithumb


Exception: No common symbols found between mercado and bithumb

In [89]:
print(df_arbitrage[['open_exchange1', 'open_exchange2', 'close_exchange1', 'close_exchange2', 'profit']])

                     open_exchange1  open_exchange2  close_exchange1  \
timestamp                                                              
2023-11-10 19:45:00             NaN             NaN              NaN   
2023-11-10 19:50:00             NaN             NaN              NaN   
2023-11-10 19:55:00             NaN             NaN              NaN   
2023-11-10 20:00:00             NaN             NaN              NaN   
2023-11-10 20:05:00             NaN             NaN              NaN   
...                             ...             ...              ...   
2023-11-11 03:40:00             NaN             NaN              NaN   
2023-11-11 03:45:00             NaN             NaN              NaN   
2023-11-11 03:50:00             NaN             NaN              NaN   
2023-11-11 03:55:00             NaN             NaN              NaN   
2023-11-11 04:00:00             NaN             NaN              NaN   

                     close_exchange2  profit  
timestamp       